Example of partial pooling from equation 12.1 of Gelman, Hill, "Data Analysis Using Regression and Multilevel/Hierarchical Models", Cambridge 2007.

In [1]:
import sympy
from sympy.stats import E, Normal

In [2]:

PerObservationSD = sympy.Symbol("PerObservationSD", positive=True)  # sd of distribution generating observations, unobserved
BetweenGaragesSD = sympy.Symbol("BetweenGaragesSD")  # how garages very from each other in expected behavior, unobserved
n_j = sympy.Symbol("n_j", positive=True)  # number of observations at the j-th garage, observed

In [3]:
neat_soln_approx = 1 / (1 + PerObservationSD**2 / (n_j * BetweenGaragesSD**2))

neat_soln_approx

1/(1 + PerObservationSD**2/(BetweenGaragesSD**2*n_j))

In [4]:
MeanGarageEffect = sympy.Symbol("MeanGarageEffect")  # center of distribution generating garages, unobserved

In [5]:
GarageEffect_j = sympy.Symbol("GarageEffect_j")  # actual expected behavior of a given garage, unobserved and the goal to estimate
GarageDistFactor_j = Normal("GarageDistFactor_j", mean=0, std=BetweenGaragesSD)
def_GarageEffect_j = MeanGarageEffect + GarageDistFactor_j

In [6]:

GarageMean_j = sympy.Symbol("GarageMean_j", mean=GarageEffect_j, std=PerObservationSD)  # mean of all observations at garage j, observed
PerObservationNoise_ji = Normal("PerObservationNoise_ji", mean=0, std = PerObservationSD / sympy.sqrt(n_j))
def_GarageMean_j = GarageEffect_j + PerObservationNoise_ji


In [7]:
n_obs = sympy.Symbol("n_j", positive=True)  # total number of observations across all garages, observed.
GrandMean = sympy.Symbol("GrandMean")  # mean of all observations, observed
ObservedMean = Normal("ObservedMean", mean=MeanGarageEffect, std = BetweenGaragesSD / sympy.sqrt(n_obs))

In [8]:
w = sympy.Symbol("w", positive=True)  # our weighting term picking how to pool specific and general observations, to solve for

In [9]:
estimate_j = sympy.Symbol("estimate_j")  # our estimate of the behavior of the j-th garage, to solve for
def_estimate_j = w * GarageMean_j + (1-w) * ObservedMean  # definition of our estimate

In [10]:
expected_error_term = GarageEffect_j - estimate_j  # square error of our estimate, to minimize


In [11]:
error_term_exact = (
    expected_error_term
        .subs(estimate_j, def_estimate_j)  # definition of estimate_j
        .subs(GarageMean_j, def_GarageMean_j)
        .subs(GarageEffect_j, def_GarageEffect_j)
).expand().simplify()


In [12]:
A = (1-w) * GarageDistFactor_j
B = w * PerObservationNoise_ji
C = (1 - w) * (ObservedMean - MeanGarageEffect)

In [13]:
assert (error_term_exact - (A - B - C)).expand() == 0

In [14]:
assert (E(C*C) - (1 - w)**2 * BetweenGaragesSD**2 / n_j).expand() == 0

In [15]:
soln_exact = sympy.solve(sympy.diff((E(A * A) + E(B * B) + E(C * C)).expand(), w), w)[0]

soln_exact

BetweenGaragesSD**2*(n_j + 1)/(BetweenGaragesSD**2*n_j + BetweenGaragesSD**2 + PerObservationSD**2)

In [16]:
neat_soln_exact = 1 / (1 + PerObservationSD**2 / ((n_j + 1) * BetweenGaragesSD**2))

neat_soln_exact

1/(1 + PerObservationSD**2/(BetweenGaragesSD**2*(n_j + 1)))

In [17]:
assert (soln_exact - neat_soln_exact).together().expand() == 0

In [18]:
error_term_approx = (
    expected_error_term
        .subs(estimate_j, def_estimate_j)  # definition of estimate_j
        .subs(ObservedMean, MeanGarageEffect)  # this step is an approximation, using the unobserved MeanGarageEffect as if it is the observed ObservedMean
        .subs(GarageMean_j, def_GarageMean_j)
        .subs(GarageEffect_j, def_GarageEffect_j)
).expand().simplify()


In [19]:
A = (1-w) * GarageDistFactor_j
B = w * PerObservationNoise_ji


In [20]:
assert (error_term_approx - (A - B)).simplify() == 0

In [21]:
soln_approx = sympy.solve(sympy.diff(E(A**2) + E(B**2), w), w)[0]

soln_approx

BetweenGaragesSD**2*n_j/(BetweenGaragesSD**2*n_j + PerObservationSD**2)

In [22]:
neat_soln_approx

1/(1 + PerObservationSD**2/(BetweenGaragesSD**2*n_j))

In [23]:
assert (soln_approx - neat_soln_approx).together().expand() == 0